# Data Cleaning

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("sba_national.csv", low_memory=False)
df.shape

(899164, 27)

## Data Validation

In [ ]:
df.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

### LoanNr_ChkDgt

### Name

### City

### State

### Zip

### Bank

### BankState

### NAICS

### ApprovalDate

### ApprovalFY

### Term

### NoEmp

### NewExist

### CreateJob

### RetainedJob

### FranchiseCode

### UrbanRural

### RevLineCr

### LowDoc

### ChgOffDate

### DisbursementDate

### DisbursementGross

### BalanceGross

### ChgOffPrinGr

### GrAppv

### SBA_Appv

### MIS_Status

### **Summary**

In [ ]:
# view and drop duplicates

In [ ]:
# drop_cols = ["LoanNr_ChkDgt", # tiene 100% de valores distintos
#              "Name",          # tiene 86.7% de valores distintos
#              "City",          # cuenta con 32581 ciudades diferentes
#              "Zip",           # cuenta con 33611 codigos postales diferentes
#              "Bank",          # cuenta con 5802 bancos diferentes
#              "ApprovalFY",    # comportamiento similar a ApprovalFY
#              "FranchiseCode"] # tiene 94.2% de codigos sin franquicia

In [ ]:
# df.shape
# df.dtypes

## Missing Values

In [ ]:
df.isnull().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

### State

### BankState

### NewExist

### RevLineCr

### LowDoc

### DisbursementDate

### MIS_Status

### **Summary**

In [ ]:
# df.shape
# df.isnull().sum()

## Outlier Detection

In [ ]:
df.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

### Term

### NoEmp

### CreateJob

### RetainedJob

### DisbursementGross

### BalanceGross

### GrAppv

### SBA_Appv

### **Summary**

In [ ]:
# df.shape